In [1]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout

In [8]:
# Carregando os dados
partidas_realizadas = pd.read_csv('../data/raw/Partidas_Realizadas.csv')
classificacao = pd.read_csv('../data/raw/Classificacao.csv')
partidas_nao_realizadas = pd.read_csv('../data/raw/Partidas_Nao_Realizadas.csv')

In [9]:
# Limpeza e pré-processamento
partidas_realizadas['Data'] = pd.to_datetime(partidas_realizadas['Data'], errors='coerce')
partidas_realizadas = partidas_realizadas[partidas_realizadas['Placar_Visitante'] != 'X']
partidas_realizadas['Placar_Mandante'] = pd.to_numeric(partidas_realizadas['Placar_Mandante'], errors='coerce')
partidas_realizadas['Placar_Visitante'] = pd.to_numeric(partidas_realizadas['Placar_Visitante'], errors='coerce')

In [10]:
# Removendo colunas irrelevantes
partidas_realizadas = partidas_realizadas.drop(columns=["Estadio", "Data", "Hora"])

In [26]:
partidas_realizadas.isnull().sum()

Rodada              0
Mandante            0
Placar_Mandante     0
Placar_Visitante    0
Visitante           0
Dia                 9
resultado           0
goleada             0
diferenca_placar    0
Resultado           0
dtype: int64

In [11]:
classificacao.head()

,Rodada,Time,Posicao,Pontos,Jogos,Vitorias,Empates,Derrotas,GP,GC,SG
0,29,Botafogo,1,60,29,18,6,5,47,25,22
1,29,Palmeiras,2,57,29,17,6,6,46,20,26
2,29,Fortaleza,3,55,29,16,7,6,38,29,9
3,29,Flamengo,4,51,28,15,6,7,45,32,13
4,29,São Paulo,5,47,29,14,5,10,38,32,6


In [12]:
partidas_nao_realizadas.head()

,Rodada,Mandante,Visitante,Data
0,28,Internacional,Grêmio,20/10
1,28,Cruzeiro,Bahia,20/10
2,28,Vitória,Bragantino,20/10
3,28,Flamengo,Fluminense,20/10
4,28,Botafogo,Criciúma,20/10


In [13]:
dias_da_semana_para_numero = { 'Segunda': 0, 'Terça': 1, 'Quarta': 2, 'Quinta': 3, 'Sexta': 4, 'Sábado': 5, 'Domingo': 6}
partidas_realizadas['Dia'] = partidas_realizadas['Dia'].replace(dias_da_semana_para_numero)
partidas_realizadas['Dia'] = pd.to_numeric(partidas_realizadas['Dia'], errors='coerce')

partidas_realizadas['Placar_Mandante'] = pd.to_numeric(partidas_realizadas['Placar_Mandante'])
partidas_realizadas['Placar_Visitante'] = pd.to_numeric(partidas_realizadas['Placar_Visitante'])

def determinar_resultado(row):
    if row['Placar_Mandante'] > row['Placar_Visitante']:
        return 0  # Ganhou
    elif row['Placar_Mandante'] < row['Placar_Visitante']:
        return 2  # Perdeu
    else:
        return 1  # Empatou

def foi_goleada(row):
    if abs(row['Placar_Mandante'] - row['Placar_Visitante']) >= 3:
        return 1  # Foi goleada
    else:
        return 0  # Não foi goleada

partidas_realizadas['resultado'] = partidas_realizadas.apply(determinar_resultado, axis=1)
partidas_realizadas['goleada'] = partidas_realizadas.apply(foi_goleada, axis=1)
partidas_realizadas['diferenca_placar'] = partidas_realizadas['Placar_Mandante'] - partidas_realizadas['Placar_Visitante']

partidas_realizadas.head()

,Rodada,Mandante,Placar_Mandante,Placar_Visitante,Visitante,Dia,resultado,goleada,diferenca_placar
0,1,Vitória,0,1,Palmeiras,6.0,2,0,-1
1,1,Internacional,2,1,Bahia,5.0,0,0,1
2,1,Criciúma,1,1,Juventude,5.0,1,0,0
3,1,Fluminense,2,2,Bragantino,5.0,1,0,0
4,1,São Paulo,1,2,Fortaleza,5.0,2,0,-1


In [14]:
classificacao['Pontos'] = classificacao['Pontos'].astype(int)
classificacao['Vitorias'] = classificacao['Vitorias'].astype(int)
classificacao['Empates'] = classificacao['Empates'].astype(int)
classificacao['Derrotas'] = classificacao['Derrotas'].astype(int)
classificacao['GP'] = classificacao['GP'].astype(int)
classificacao['GC'] = classificacao['GC'].astype(int)
classificacao['SG'] = classificacao['GP'] - classificacao['GC']  # Saldo de Gols
classificacao['Jogos'] = classificacao['Jogos'].astype(int)

In [15]:
print(partidas_realizadas.columns)

Index(['Rodada', 'Mandante', 'Placar_Mandante', 'Placar_Visitante',
       'Visitante', 'Dia', 'resultado', 'goleada', 'diferenca_placar'],
      dtype='object')


In [16]:
partidas_realizadas['Placar_Mandante'] = pd.to_numeric(partidas_realizadas['Placar_Mandante'], errors='coerce')
partidas_realizadas['Placar_Visitante'] = pd.to_numeric(partidas_realizadas['Placar_Visitante'], errors='coerce')
partidas_realizadas = partidas_realizadas.dropna(subset=['Placar_Mandante', 'Placar_Visitante'])

In [17]:
partidas_realizadas['Resultado'] = np.where(partidas_realizadas['Placar_Mandante'] > partidas_realizadas['Placar_Visitante'], 'Mandante',
                                           np.where(partidas_realizadas['Placar_Mandante'] < partidas_realizadas['Placar_Visitante'], 'Visitante', 'Empate'))

In [18]:
X = partidas_realizadas[['Mandante', 'Visitante', 'Rodada']]
y = partidas_realizadas['Resultado']

In [19]:
X = pd.get_dummies(X, columns=['Mandante', 'Visitante'], drop_first=True)

In [20]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [21]:
X = X.astype(np.float32)
y_categorical = y_categorical.astype(np.float32)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.3, random_state=42)

In [23]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))  # 3 classes: 'Mandante', 'Visitante', 'Empate'

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [34]:
model.fit(X_train, y_train, epochs=150, batch_size=10, validation_data=(X_test, y_test))

ValueError: object __array__ method not producing an array

In [30]:
X_nao_realizadas = partidas_nao_realizadas[['Mandante', 'Visitante', 'Rodada']]
X_nao_realizadas = pd.get_dummies(X_nao_realizadas, columns=['Mandante', 'Visitante'], drop_first=True)
X_nao_realizadas = X_nao_realizadas.reindex(columns=X.columns, fill_value=0).astype(np.float32)

In [31]:
classificacao = classificacao.drop_duplicates(subset='Time').reset_index(drop=True)

predicted_results = model.predict(X_nao_realizadas)
predicted_classes = np.argmax(predicted_results, axis=1)
partidas_nao_realizadas['Resultado'] = label_encoder.inverse_transform(predicted_classes)

ValueError: object __array__ method not producing an array

In [35]:
print(partidas_nao_realizadas)

    Rodada       Mandante      Visitante   Data
0       28  Internacional         Grêmio  20/10
1       28       Cruzeiro          Bahia  20/10
2       28        Vitória     Bragantino  20/10
3       28       Flamengo     Fluminense  20/10
4       28       Botafogo       Criciúma  20/10
..     ...            ...            ...    ...
85      36      Palmeiras     Fluminense  08/12
86      36     Bragantino       Criciúma  08/12
87      36      Fortaleza  Internacional  08/12
88      36         Cuiabá          Vasco  08/12
89      36      Juventude       Cruzeiro  08/12

[90 rows x 4 columns]


In [ ]:
for index, row in partidas_nao_realizadas.iterrows():
    mandante = row['Mandante']
    visitante = row['Visitante']
    resultado = row['Resultado']

    mandante_row = classificacao.loc[classificacao['Time'] == mandante].index[0]
    visitante_row = classificacao.loc[classificacao['Time'] == visitante].index[0]

    if resultado == 'Mandante':
        classificacao.loc[mandante_row, 'Pontos'] += 3
        classificacao.loc[mandante_row, 'Vitorias'] += 1
        classificacao.loc[visitante_row, 'Derrotas'] += 1
    elif resultado == 'Visitante':
        classificacao.loc[visitante_row, 'Pontos'] += 3
        classificacao.loc[visitante_row, 'Vitorias'] += 1
        classificacao.loc[mandante_row, 'Derrotas'] += 1
    else:
        classificacao.loc[mandante_row, 'Pontos'] += 1
        classificacao.loc[visitante_row, 'Pontos'] += 1
        classificacao.loc[mandante_row, 'Empates'] += 1
        classificacao.loc[visitante_row, 'Empates'] += 1
    classificacao.loc[mandante_row, 'Jogos'] += 1
    classificacao.loc[visitante_row, 'Jogos'] += 1

In [ ]:
classificacao = classificacao.sort_values(by=['Pontos', 'Vitorias', 'SG'], ascending=False).reset_index(drop=True)
classificacao['Posicao'] = classificacao.index + 1

print(classificacao[['Time', 'Posicao', 'Pontos', 'Jogos', 'Vitorias', 'Empates', 'Derrotas']])